In [7]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import os

os.chdir("C:/Users/santi/GNN4OPF")

import funciones.parser  as ps
path_datos = './data/prueba_sin_sobrecargas.raw'

In [29]:
df = ps.branch_data_df('data/prueba_sin_sobrecargas.raw')
df

,I,J,R,X,B,BASKV
0,ACO150_B,PBE150BX,0.002933,0.009957,0.00376,150
1,ACO150_B,PBE150RX,0.002933,0.009957,0.00376,150
2,ARA150_B,CLFAUX150_B,0.015066,0.045404,0.01581,150
3,ARA150_B,TGO150_B,0.033038,0.099566,0.03467,150
4,ARB150P1,ARB150_B,0.000920,0.003720,0.00123,150
...,...,...,...,...,...,...
138,SCA150_B,SCAROC_X,0.003609,0.013645,0.00517,150
139,SGDE.ARG,SU5500_B,0.000040,0.000410,0.03890,500
140,SJ5500_B,SU5500_B,0.001710,0.016060,1.52925,500
141,TAB150_B,TAC150_B,0.007609,0.022935,0.00797,150


In [30]:
data = ps.data(path_datos)
data

,BASKV,VM,VA,IDE,V (KV),SL,SG
NAME,,,,,,,
ACO150_B,150.0,1.03751,-5.5996,1,154.883863-15.185404j,16.148+7.872j,0.0+0.0j
ARA150_B,150.0,1.05099,1.1028,1,157.619299+3.034151j,0.903+0.029j,0.0+0.0j
ARB150P1,150.0,1.11299,-6.1448,1,165.989306-17.870421j,0.000+0.000j,0.0+0.0j
ARB150P2,150.0,1.11261,-6.1011,1,165.946210-17.737756j,0.000+0.000j,0.0+0.0j
ARB150_B,150.0,1.11280,-6.1230,2,165.967756-17.804224j,0.000+0.000j,0.0+0.0j
...,...,...,...,...,...,...,...
TGO150_B,150.0,1.06464,0.8698,1,159.677599+2.424232j,1.001-6.298j,0.0+0.0j
TRI150_B,150.0,1.03032,-5.0595,2,153.945827-13.629621j,7.994+0.842j,0.0+0.0j
TYT150_B,150.0,1.11525,-6.9249,1,166.067144-20.169564j,11.506-4.545j,0.0+0.0j


In [31]:
def get_Admittances(branch_df):
  branch_df['Ybase'] = 100*1000000/((branch_df['BASKV']*1000)**2)
  
  Admitance = np.empty((branch_df.shape[0],2),dtype = 'complex_')
  Admitance[:,0] = (1/(branch_df['R']+branch_df['X']*1j))*branch_df['Ybase']
  Admitance[:,1] = branch_df['B']/2*1j*branch_df['Ybase']
  Admitance = pd.DataFrame(Admitance,columns=['y','ysh'])

  Admitance['I'] = branch_df['I']
  Admitance['J'] = branch_df['J']
  Admitance = Admitance[['I','J','y','ysh']]
  return Admitance

def get_graph(edges_df):
  new_rows = []
  for _, row in edges_df.iterrows():
      source, sink, _, self_loop_weight = row
      new_rows.append({'source': source, 'target': source, 'weight': self_loop_weight})
      new_rows.append({'source': sink, 'target': sink, 'weight': self_loop_weight})
  new_edges_df = edges_df.drop(columns=['ysh'])
  new_edges_df.rename(columns={'I': 'source', 'J': 'target','y': 'weight'}, inplace=True)
  new_rows = pd.DataFrame(new_rows)
  new_edges_df = pd.concat([new_edges_df,new_rows], axis=0)
  new_edges_df_2 = new_edges_df.groupby(['source', 'target'])['weight'].sum().reset_index()
  G = nx.from_pandas_edgelist(new_edges_df_2, edge_attr=True)
  return G

def get_Ybus(G):
  Y = nx.laplacian_matrix(G).toarray()
  for i, node in enumerate(G.nodes()):
      degree = G.degree(node,weight='weight')
      Y[i, i] += (degree - Y[i, i])  / 2
  return Y

In [32]:
Admitance = get_Admittances(df)
G = get_graph(Admitance)
Ybus = get_Ybus(G)
Admitance
# Ybus
# for edge in G.edges():
#     weight_complex = G.edges[edge]['weight']
#     weight_magnitude = abs(weight_complex)
#     G.edges[edge]['weight'] = weight_magnitude

# # Definir el layout y visualizar los nodos
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_size=1000, node_color='skyblue', font_size=10, font_color='darkblue', font_weight='bold')

# # Dibujar los enlaces con colores y grosores basados en los valores transformados
# nx.draw_networkx_edges(G, pos, edge_color='gray')

# # Agregar anotaciones o leyendas si es necesario

# # Mostrar la visualización
# plt.show()


,I,J,y,ysh
0,ACO150_B,PBE150BX,2.6551120-9.0143580j,0.000000+0.000183j
1,ACO150_B,PBE150RX,2.6551120-9.0143580j,0.000000+0.000183j
2,ARA150_B,CLFAUX150_B,0.6421800-1.9352960j,0.000000+0.000771j
3,ARA150_B,TGO150_B,0.2928480-0.8825370j,0.000000+0.001691j
4,ARB150P1,ARB150_B,6.111202-024.7105130j,0.000000+0.000060j
...,...,...,...,...
138,SCA150_B,SCAROC_X,1.7669140-6.6812670j,0.000000+0.000252j
139,SGDE.ARG,SU5500_B,26.427953-270.886519j,0.000000+0.002181j
140,SJ5500_B,SU5500_B,0.7350120-6.9030960j,0.000000+0.085730j
141,TAB150_B,TAC150_B,1.2712080-3.8314110j,0.000000+0.000389j


In [33]:
Ybus

array([[ 5.31022443 -18.02834887j, -2.65511221  +9.01435782j,
        -2.65511221  +9.01435782j, ...,  0.          +0.j        ,
         0.          +0.j        ,  0.          +0.j        ],
       [-2.65511221  +9.01435782j,  3.24015337 -10.98920802j,
         0.          +0.j        , ...,  0.          +0.j        ,
         0.          +0.j        ,  0.          +0.j        ],
       [-2.65511221  +9.01435782j,  0.          +0.j        ,
         3.27985788 -11.12336248j, ...,  0.          +0.j        ,
         0.          +0.j        ,  0.          +0.j        ],
       ...,
       [ 0.          +0.j        ,  0.          +0.j        ,
         0.          +0.j        , ...,  0.25261224  -0.75703971j,
        -0.25261224  +0.75900769j,  0.          +0.j        ],
       [ 0.          +0.j        ,  0.          +0.j        ,
         0.          +0.j        , ..., -0.25261224  +0.75900769j,
         2.01952584  -7.43805417j,  0.          +0.j        ],
       [ 0.          +0.j   

In [34]:
# se obtiene un orden que se usara para ordenar luego el voltaje
nombres_ordenados = list(G.nodes)
nombres_ordenados

['ACO150_B',
 'PBE150BX',
 'PBE150RX',
 'ARA150_B',
 'CLFAUX150_B',
 'TGO150_B',
 'ARB150P1',
 'ARB150_B',
 'TYT150_B',
 'ARB150P2',
 'MEB150P1',
 'ART150_B',
 'JPT_150B',
 'RIV150_B',
 'BAY150_B',
 'BON150_B',
 'PAL150_B',
 'TRI150_B',
 'BIF150_B',
 'MVA150_B',
 'PAN150_F',
 'PAZ1501B',
 'PAZ1502B',
 'BOB150_B',
 'YOU150_B',
 'DUR1501F',
 'DUR1502F',
 'PAM150_B',
 'VAL150_B',
 'BRU500_B',
 'MVB500_B',
 'PAL500_B',
 'C.ELIA',
 'SGDE.ARG',
 'SJ5500_B',
 'CGR150_B',
 'MAL150_B',
 'PES150_B',
 'SU5150_B',
 'COE1502P',
 'COL150_B',
 'CON150_B',
 'LIB150_B',
 'MEN150_B',
 'NPA150_B',
 'DOL150_B',
 'MER150_B',
 'PIA150_B',
 'EFI150_X',
 'SVA150_B',
 'EMA150_B',
 'FBE150_B',
 'FBEMER_X',
 'FBESJA_X',
 'SJA150_B',
 'FLO150_B',
 'POL150_B',
 'PRO1501X',
 'PRO1502X',
 'LAD150_B',
 'MEB150_B',
 'MA5500_B',
 'MI5500_B',
 'PAZMAL_X',
 'SCAMAL1X',
 'SCAMAL2X',
 'MDI150_X',
 'STE150_B',
 'TAC150_B',
 'ME5500_B',
 'SC5500_B',
 'MEB150P2',
 'MEL150_B',
 'TAB150_B',
 'ROS150_B',
 'MVB150_B',
 'MVLMVA_X'

In [37]:
data = data.reindex(nombres_ordenados)
V = data['V (KV)'].to_numpy()*1000
diagV = np.diag(V)
V

array([154883.86298117-15185.40385639j, 155075.65741588-14775.85060263j,
       154888.59675979-15412.15935657j, 157619.29920405 +3034.15073383j,
       156655.69444478 +3870.44057884j, 159677.59866873 +2424.23171096j,
       165989.30599243-17870.42104714j, 165967.75593359-17804.22394741j,
       166067.14409054-20169.5639482j , 165946.21049342-17737.75620321j,
       165690.30905177-16910.26573797j, 162071.75462105 +4015.85810293j,
       161980.31006721 +6043.16138557j, 162333.77377395-13334.28560363j,
       155556.14228347-14408.32682461j, 158500.03743462-18909.24401512j,
       148550.44362751 -4643.47144604j, 153945.82724731-13629.6213132j ,
       155423.14401857-16476.0656524j , 156569.50278204-13200.62209081j,
       156178.26397763-14554.75475345j, 155337.31868259-17361.28292358j,
       155421.81814252-17735.83309355j, 158489.78338936-18931.86819736j,
       155385.91288944-20439.83795839j, 157295.7457296 -20883.5186539j ,
       158357.02307902-19081.42415548j, 161629.2437

In [23]:
data


,BASKV,VM,VA,IDE,V (KV),SL,SG
NAME,,,,,,,
ACO150_B,150.0,1.03751,-5.5996,1,154.883863-15.185404j,16.148+7.872j,0.0000+0.0000j
PBE150BX,150.0,1.03852,-5.4428,1,155.075657-14.775851j,0.000+0.000j,0.0000+0.0000j
PBE150RX,150.0,1.03769,-5.6825,1,154.888597-15.412159j,0.000+0.000j,0.0000+0.0000j
ARA150_B,150.0,1.05099,1.1028,1,157.619299+3.034151j,0.903+0.029j,0.0000+0.0000j
CLFAUX150_B,150.0,1.04469,1.4153,2,156.655694+3.870441j,0.000+0.000j,70.0000+5.7080j
...,...,...,...,...,...,...,...
SCA150_B,150.0,1.04490,-6.2282,2,155.809901-17.003968j,4.373+4.941j,0.0000+0.0000j
PIB150_B,150.0,1.05811,-6.8640,2,157.578920-18.968693j,0.000+0.000j,0.0000+0.0000j
ROC150_B,150.0,1.04531,-6.9013,2,155.660449-18.840568j,8.869-0.814j,0.0000+0.0000j


In [38]:
h = np.matmul(diagV,Ybus.conjugate()).dot(V.conjugate())
h

array([-3.55569273e+08-1.75152478e+08j,  7.91979816e+05+1.55761085e+06j,
        5.61942504e+05+1.29646695e+06j, -1.68965186e+08-3.03162473e+05j,
        1.53647822e+09+1.25567475e+08j, -4.36138362e+08+9.73514670e+07j,
        1.57307992e+05+1.90254434e+05j, -1.08722211e+06-2.96780542e+06j,
       -2.52661601e+08+9.95225915e+07j,  9.29795091e+05+2.76874759e+06j,
        5.18451126e+05+1.76524888e+06j, -1.88460626e+08-3.05877596e+07j,
        1.53644882e+09-3.15342113e+08j, -4.80535277e+08-1.17270668e+08j,
        6.10881895e+04+6.91085063e+04j, -8.91894828e+07+1.11846284e+07j,
        1.17191936e+09-1.61615827e+09j, -1.75512327e+08-1.85764605e+07j,
       -5.26567295e+08-1.32563603e+08j,  7.57928212e+09+1.43519824e+09j,
        7.99616166e+04+1.14058680e+05j, -2.87016625e+08-2.55254153e+07j,
       -2.05935838e+05-5.25901650e+05j, -3.63074748e+06-1.09997971e+07j,
       -4.94460103e+08+1.97674086e+05j, -4.59345792e+08-6.72442430e+07j,
        5.33320268e+04+3.87866776e+04j,  4.45721493

In [39]:
Volt = 150000*1.03571
angle = -5.5996*2*np.pi/360
Voltaje = Volt*np.cos(angle) + Volt*np.sin(angle)*1j
Potencia = -(16.148 + 7.872*1j)*1000000

(154615.1513992371-15159.058349415573j)

In [40]:
Volt_2 = 150000*1.03852
angle_2 = -5.4428*2*np.pi/360
Voltaje_2 = Volt*np.cos(angle) + Volt*np.sin(angle)*1j
Potencia_2 = 0


In [41]:
Volt_3 = 150000*1.03769
angle_3 = -5.6825*np.pi/360
Voltaje_3 = Volt*np.cos(angle) + Volt*np.sin(angle)*1j
Potencia_3 = 0

In [48]:
Ybase = 100*1e6/((150e3)**2)
Ysh12 = (1j*0.00376/2)*Ybase
Ysh13 = 1j *0.00376/2*Ybase
Y12 = 1/(2.93284e-3+(9.95727e-3)*1j)*Ybase
Y13 = Y12
I = Volt*(Ysh12+Ysh13)-Volt_2*Y12-Volt_3*Y13
S = Volt*I.conjugate()
S/1000000
Y12



(0.12097440815396449-0.41071959093548444j)